In [1]:
import pandas as pd
import glob
import plotly
import seaborn as sns
from matplotlib import pyplot as plt
import matplotlib.ticker as tkr
import numpy as np
%matplotlib inline

#### 1. Leitura e limpeza

In [2]:
def find_fpaths(dir_path, pattern):
    '''
    >> DESCRIÇÃO
    
    Usa o módulo glob para buscar todos os arquivos
    que correspondam ao padrão passado na variável 
    pattern'. Retorna uma lista de paths no formato 
    string. 
    
    >> PARÂMETROS
    
    dir_path -> uma string com o caminho para o
    diretório onde a busca pelos arquivos será
    realizada.
    
    pattern -> uma string com o padrão de texto
    que deve ser procurado no diretório.
    '''  
    # Caso o dir_path não possua final em "/", adicone
    if dir_path[-1] != '/':
        dir_path += "/"
    
    full_pattern = dir_path + pattern
    files = glob.glob(full_pattern)
    
    return files

In [3]:
def read_and_concat(data_, axis=0, sep=None, quotechar=None, ignore_index=True):  
    '''
    >> DESCRIÇÃO
    
    Chama a função find_fpaths definida acima
    e funções built-in do pandas para concatenar
    todos os arquivos em formato .csv que estão
    dentro de um diretório. Retorna um dataframe.
    
    >> PARÂMETROS
    
    data_ -> Uma lista de caminhos para arquivos .csv
    
    axis -> eixo em que vai acontecer a concatenação (default: 0)
    
    sep -> separador dos arquivos .csv em questão (default: ',')
    
    quotechar -> especifica o delimitador que encapsula
    os campos do arquivo (default: None)
    
    ignore_index -> especifica se o índice dos arquivos
    deve ser descartado ao concatenar (default: True)
    '''
    
    # Lê todos os arquivos em uma lista de dataframes
    if quotechar:
        data_ = [ pd.read_csv( file, sep = sep, quotechar = quotechar ) for file in data_ ]
    else:
        data_ = [ pd.read_csv( file, sep = sep ) for file in data_ ]

    
    # Concatena todos em um único dataframe
    data_ = pd.concat(data_, axis = axis, ignore_index = ignore_index)
    
    # Retorna
    return data_

In [4]:
csv_data = []
# Lê apenas os anos entre 2003 e 2018, únicos com dados confiáveis
for year in range(2003, 2019):
    csv_data.extend(
        find_fpaths(dir_path = "../data/csvs/" + str(year) + "/", pattern = "*.csv")
    )

In [5]:
# Concatena tudo em um único dataframe
df = read_and_concat( csv_data, sep = ',' )

In [6]:
# Remove colunas com NaN, que ocorrem quando não houve orientação do governo
df = df.dropna( subset = [ "com_governo", "contra_governo" ] )

In [7]:
# A função abaixo substitui a sigla de partidos
# que mudaram de nome ou que estão com padronização
# problemática pelos valores certos.
def replace_parties(row):
    
    correspondence = {
        "PFL"       :"DEM",
        "PMDB"      :"MDB",
        "PEN"       :"PATRI",
        "PATRI"     :"PATRIOTA",
        "Patriota"  :"PATRI",
        "Avante"    :"AVANTE",
        "PTdoB"     :"AVANTE",
        "PMR"       :"PRB",
        "PSDC"      :"DC",
        "PTN"       :"PODE",
        "Podemos"   :"PODE",
        "S.Part."   :"SEM-PARTIDO",
        "SDD"       :"SD",
        "Solidaried":"SD",
    }
    
    for key, value in correspondence.items():
        if row.siglaPartido == key:
            siglaPartido = value
            return pd.Series({
                'siglaPartido':siglaPartido
            })
        
    siglaPartido = row.siglaPartido
    return pd.Series({
        'siglaPartido':siglaPartido
    })

In [8]:
df['siglaPartido'] = df.apply(replace_parties, axis=1)

In [9]:
# Coloca os ids como tipo string
df['voteId']         = df.voteId.astype(str)
df['idProposicao']   = df.idProposicao.astype(str)
df['dataHoraInicio'] = pd.to_datetime(df.dataHoraInicio)
df['dataHoraFim']    = pd.to_datetime(df.dataHoraFim)

In [10]:
# Há algumas votações cruciais que não estão na API do Câmara. Vamos adicionar TRÊS delas manualmente: impeachment de Dilma e denúncias rejeitadas contra Temer
new_rows = [
  # Impeachment
  {
      'siglaPartido':'DEM', 
       'voteId':'0001', 
       'idProposicao':'0001', 
       'dataHoraInicio':pd.to_datetime("2016-04-17 14:00:00.0"), 
       'dataHoraFim':pd.to_datetime("2016-04-17 23:50:00.0"),
       'com_governo':0,
       'contra_governo':28,
  },
  {
       'siglaPartido':'PCdoB', 
       'voteId':'0001', 
       'idProposicao':'0001', 
       'dataHoraInicio':pd.to_datetime("2016-04-17 14:00:00.0"), 
       'dataHoraFim':pd.to_datetime("2016-04-17 23:50:00.0"),
       'com_governo':10,
       'contra_governo':0,
  },
  {
       'siglaPartido':'PDT',
       'voteId':'0001', 
       'idProposicao':'0001', 
       'dataHoraInicio':pd.to_datetime("2016-04-17 14:00:00.0"), 
       'dataHoraFim':pd.to_datetime("2016-04-17 23:50:00.0"),
       'com_governo':12,
       'contra_governo':7,
  },
  {
       'siglaPartido':'PATRI', 
       'voteId':'0001', 
       'idProposicao':'0001', 
       'dataHoraInicio':pd.to_datetime("2016-04-17 14:00:00.0"), 
       'dataHoraFim':pd.to_datetime("2016-04-17 23:50:00.0"),
       'com_governo':1,
       'contra_governo':1,
  },
  {
       'siglaPartido':'PHS', 
       'voteId':'0001', 
       'idProposicao':'0001', 
       'dataHoraInicio':pd.to_datetime("2016-04-17 14:00:00.0"), 
       'dataHoraFim':pd.to_datetime("2016-04-17 23:50:00.0"),
       'com_governo':1,
       'contra_governo':7,
  },
  {
       'siglaPartido':'PMB', 
       'voteId':'0001', 
       'idProposicao':'0001', 
       'dataHoraInicio':pd.to_datetime("2016-04-17 14:00:00.0"), 
       'dataHoraFim':pd.to_datetime("2016-04-17 23:50:00.0"),
       'com_governo':0,
       'contra_governo':1,
  },
  {
       'siglaPartido':'MDB', 
       'voteId':'0001', 
       'idProposicao':'0001', 
       'dataHoraInicio':pd.to_datetime("2016-04-17 14:00:00.0"), 
       'dataHoraFim':pd.to_datetime("2016-04-17 23:50:00.0"),
       'com_governo':7,
       'contra_governo':59,
  },
  {
       'siglaPartido':'PP', 
       'voteId':'0001', 
       'idProposicao':'0001', 
       'dataHoraInicio':pd.to_datetime("2016-04-17 14:00:00.0"), 
       'dataHoraFim':pd.to_datetime("2016-04-17 23:50:00.0"),
       'com_governo':4,
       'contra_governo':41,
  },
  {
       'siglaPartido':'PPS', 
       'voteId':'0001', 
       'idProposicao':'0001', 
       'dataHoraInicio':pd.to_datetime("2016-04-17 14:00:00.0"), 
       'dataHoraFim':pd.to_datetime("2016-04-17 23:50:00.0"),
       'com_governo':0,
       'contra_governo':8,
  },
  {
       'siglaPartido':'PR', 
       'voteId':'0001', 
       'idProposicao':'0001', 
       'dataHoraInicio':pd.to_datetime("2016-04-17 14:00:00.0"), 
       'dataHoraFim':pd.to_datetime("2016-04-17 23:50:00.0"),
       'com_governo':10,
       'contra_governo':29,
  },
  {
       'siglaPartido':'PRB', 
       'voteId':'0001', 
       'idProposicao':'0001', 
       'dataHoraInicio':pd.to_datetime("2016-04-17 14:00:00.0"), 
       'dataHoraFim':pd.to_datetime("2016-04-17 23:50:00.0"),
       'com_governo':0,
       'contra_governo':22,
  },
  {
       'siglaPartido':'PROS', 
       'voteId':'0001', 
       'idProposicao':'0001', 
       'dataHoraInicio':pd.to_datetime("2016-04-17 14:00:00.0"), 
       'dataHoraFim':pd.to_datetime("2016-04-17 23:50:00.0"),
       'com_governo':2,
       'contra_governo':4,
  },
  {
       'siglaPartido':'PSB', 
       'voteId':'0001', 
       'idProposicao':'0001', 
       'dataHoraInicio':pd.to_datetime("2016-04-17 14:00:00.0"), 
       'dataHoraFim':pd.to_datetime("2016-04-17 23:50:00.0"),
       'com_governo':3,
       'contra_governo':29,
  },
  {
       'siglaPartido':'PSC', 
       'voteId':'0001', 
       'idProposicao':'0001', 
       'dataHoraInicio':pd.to_datetime("2016-04-17 14:00:00.0"), 
       'dataHoraFim':pd.to_datetime("2016-04-17 23:50:00.0"),
       'com_governo':0,
       'contra_governo':10,
  },
  {
       'siglaPartido':'PSD', 
       'voteId':'0001', 
       'idProposicao':'0001', 
       'dataHoraInicio':pd.to_datetime("2016-04-17 14:00:00.0"), 
       'dataHoraFim':pd.to_datetime("2016-04-17 23:50:00.0"),
       'com_governo':8,
       'contra_governo':29,
  },
  {
       'siglaPartido':'PSDB', 
       'voteId':'0001', 
       'idProposicao':'0001', 
       'dataHoraInicio':pd.to_datetime("2016-04-17 14:00:00.0"), 
       'dataHoraFim':pd.to_datetime("2016-04-17 23:50:00.0"),
       'com_governo':0,
       'contra_governo':52,
  },
  {
       'siglaPartido':'PSL', 
       'voteId':'0001', 
       'idProposicao':'0001', 
       'dataHoraInicio':pd.to_datetime("2016-04-17 14:00:00.0"), 
       'dataHoraFim':pd.to_datetime("2016-04-17 23:50:00.0"),
       'com_governo':0,
       'contra_governo':2,
  },
  {
       'siglaPartido':'PSOL', 
       'voteId':'0001', 
       'idProposicao':'0001', 
       'dataHoraInicio':pd.to_datetime("2016-04-17 14:00:00.0"), 
       'dataHoraFim':pd.to_datetime("2016-04-17 23:50:00.0"),
       'com_governo':6,
       'contra_governo':0,
  },
  {
       'siglaPartido':'PT', 
       'voteId':'0001', 
       'idProposicao':'0001', 
       'dataHoraInicio':pd.to_datetime("2016-04-17 14:00:00.0"), 
       'dataHoraFim':pd.to_datetime("2016-04-17 23:50:00.0"),
       'com_governo':60,
       'contra_governo':0,
  },
  {
       'siglaPartido':'AVANTE', 
       'voteId':'0001', 
       'idProposicao':'0001', 
       'dataHoraInicio':pd.to_datetime("2016-04-17 14:00:00.0"), 
       'dataHoraFim':pd.to_datetime("2016-04-17 23:50:00.0"),
       'com_governo':1,
       'contra_governo':2,
  },
  {
       'siglaPartido':'PTB', 
       'voteId':'0001', 
       'idProposicao':'0001', 
       'dataHoraInicio':pd.to_datetime("2016-04-17 14:00:00.0"), 
       'dataHoraFim':pd.to_datetime("2016-04-17 23:50:00.0"),
       'com_governo':6,
       'contra_governo':14,
  },
  {
       'siglaPartido':'PTN', 
       'voteId':'0001', 
       'idProposicao':'0001', 
       'dataHoraInicio':pd.to_datetime("2016-04-17 14:00:00.0"), 
       'dataHoraFim':pd.to_datetime("2016-04-17 23:50:00.0"),
       'com_governo':4,
       'contra_governo':8,
  },
  {
       'siglaPartido':'PV', 
       'voteId':'0001', 
       'idProposicao':'0001', 
       'dataHoraInicio':pd.to_datetime("2016-04-17 14:00:00.0"), 
       'dataHoraFim':pd.to_datetime("2016-04-17 23:50:00.0"),
       'com_governo':0,
       'contra_governo':6,
  },
  {
       'siglaPartido':'REDE', 
       'voteId':'0001', 
       'idProposicao':'0001', 
       'dataHoraInicio':pd.to_datetime("2016-04-17 14:00:00.0"), 
       'dataHoraFim':pd.to_datetime("2016-04-17 23:50:00.0"),
       'com_governo':2,
       'contra_governo':2,
  },
  {
       'siglaPartido':'SD', 
       'voteId':'0001', 
       'idProposicao':'0001', 
       'dataHoraInicio':pd.to_datetime("2016-04-17 14:00:00.0"), 
       'dataHoraFim':pd.to_datetime("2016-04-17 23:50:00.0"),
       'com_governo':0,
       'contra_governo':14,
  },
  # Temer - 1
  {
      'siglaPartido':'DEM', 
       'voteId':'0002', 
       'idProposicao':'0002',
       'dataHoraInicio':pd.to_datetime("2017-08-02 09:00:00.0"),
       'dataHoraFim':pd.to_datetime("2017-08-02 13:54:00.0"),
       'com_governo':7,
       'contra_governo':23,
  },
  {
       'siglaPartido':'PCdoB', 
       'voteId':'0002', 
       'idProposicao':'0002',
       'dataHoraInicio':pd.to_datetime("2017-08-02 09:00:00.0"),
       'dataHoraFim':pd.to_datetime("2017-08-02 13:54:00.0"),
       'com_governo':10,
       'contra_governo':0,
  },
  {
       'siglaPartido':'PDT',       
       'voteId':'0002',
       'idProposicao':'0002', 
       'dataHoraInicio':pd.to_datetime("2017-08-02 09:00:00.0"), 
       'dataHoraFim':pd.to_datetime("2017-08-02 13:54:00.0"),
       'com_governo':1,
       'contra_governo':17,
  },
  {
       'siglaPartido':'PATRI', 
       'voteId':'0002', 
       'idProposicao':'0002',
       'dataHoraInicio':pd.to_datetime("2017-08-02 09:00:00.0"),
       'dataHoraFim':pd.to_datetime("2017-08-02 13:54:00.0"),
       'com_governo':3,
       'contra_governo':0,
  },
  {
       'siglaPartido':'PHS', 
       'voteId':'0002', 
       'idProposicao':'0002',
       'dataHoraInicio':pd.to_datetime("2017-08-02 09:00:00.0"),
       'dataHoraFim':pd.to_datetime("2017-08-02 13:54:00.0"),
       'com_governo':1,
       'contra_governo':6,
  },
  {
       'siglaPartido':'PMB', 
       'voteId':'0002', 
       'idProposicao':'0002',
       'dataHoraInicio':pd.to_datetime("2017-08-02 09:00:00.0"),
       'dataHoraFim':pd.to_datetime("2017-08-02 13:54:00.0"),
       'com_governo':0,
       'contra_governo':1,
  },
  {
       'siglaPartido':'MDB', 
       'voteId':'0002', 
       'idProposicao':'0002',
       'dataHoraInicio':pd.to_datetime("2017-08-02 09:00:00.0"),
       'dataHoraFim':pd.to_datetime("2017-08-02 13:54:00.0"),
       'com_governo':52,
       'contra_governo':7,
  },
  {
       'siglaPartido':'PP', 
       'voteId':'0002', 
       'idProposicao':'0002',
       'dataHoraInicio':pd.to_datetime("2017-08-02 09:00:00.0"),
       'dataHoraFim':pd.to_datetime("2017-08-02 13:54:00.0"),
       'com_governo':37,
       'contra_governo':8,
  },
  {
       'siglaPartido':'PPS', 
       'voteId':'0002', 
       'idProposicao':'0002',
       'dataHoraInicio':pd.to_datetime("2017-08-02 09:00:00.0"),
       'dataHoraFim':pd.to_datetime("2017-08-02 13:54:00.0"),
       'com_governo':1,
       'contra_governo':9,
  },
  {
       'siglaPartido':'PR', 
       'voteId':'0002', 
       'idProposicao':'0002',
       'dataHoraInicio':pd.to_datetime("2017-08-02 09:00:00.0"),
       'dataHoraFim':pd.to_datetime("2017-08-02 13:54:00.0"),
       'com_governo':28,
       'contra_governo':9,
  },
  {
       'siglaPartido':'PRB', 
       'voteId':'0002', 
       'idProposicao':'0002',
       'dataHoraInicio':pd.to_datetime("2017-08-02 09:00:00.0"),
       'dataHoraFim':pd.to_datetime("2017-08-02 13:54:00.0"),
       'com_governo':15,
       'contra_governo':7,
  },
  {
       'siglaPartido':'PROS', 
       'voteId':'0002', 
       'idProposicao':'0002',
       'dataHoraInicio':pd.to_datetime("2017-08-02 09:00:00.0"),
       'dataHoraFim':pd.to_datetime("2017-08-02 13:54:00.0"),
       'com_governo':3,
       'contra_governo':2,
  },
  {
       'siglaPartido':'PSB', 
       'voteId':'0002', 
       'idProposicao':'0002',
       'dataHoraInicio':pd.to_datetime("2017-08-02 09:00:00.0"),
       'dataHoraFim':pd.to_datetime("2017-08-02 13:54:00.0"),
       'com_governo':11,
       'contra_governo':22,
  },
  {
       'siglaPartido':'PSC', 
       'voteId':'0002', 
       'idProposicao':'0002',
       'dataHoraInicio':pd.to_datetime("2017-08-02 09:00:00.0"),
       'dataHoraFim':pd.to_datetime("2017-08-02 13:54:00.0"),
       'com_governo':5,
       'contra_governo':4,
  },
  {
       'siglaPartido':'PSD', 
       'voteId':'0002', 
       'idProposicao':'0002',
       'dataHoraInicio':pd.to_datetime("2017-08-02 09:00:00.0"),
       'dataHoraFim':pd.to_datetime("2017-08-02 13:54:00.0"),
       'com_governo':22,
       'contra_governo':14,
  },
  {
       'siglaPartido':'PSDB', 
       'voteId':'0002', 
       'idProposicao':'0002',
       'dataHoraInicio':pd.to_datetime("2017-08-02 09:00:00.0"),
       'dataHoraFim':pd.to_datetime("2017-08-02 13:54:00.0"),
       'com_governo':22,
       'contra_governo':21,
  },
  {
       'siglaPartido':'PSL', 
       'voteId':'0002', 
       'idProposicao':'0002',
       'dataHoraInicio':pd.to_datetime("2017-08-02 09:00:00.0"),
       'dataHoraFim':pd.to_datetime("2017-08-02 13:54:00.0"),
       'com_governo':3,
       'contra_governo':0,
  },
  {
       'siglaPartido':'PSOL', 
       'voteId':'0002', 
       'idProposicao':'0002',
       'dataHoraInicio':pd.to_datetime("2017-08-02 09:00:00.0"),
       'dataHoraFim':pd.to_datetime("2017-08-02 13:54:00.0"),
       'com_governo':0,
       'contra_governo':6,
  },
  {
       'siglaPartido':'PT', 
       'voteId':'0002', 
       'idProposicao':'0002',
       'dataHoraInicio':pd.to_datetime("2017-08-02 09:00:00.0"),
       'dataHoraFim':pd.to_datetime("2017-08-02 13:54:00.0"),
       'com_governo':0,
       'contra_governo':58,
  },
  {
       'siglaPartido':'AVANTE', 
       'voteId':'0002', 
       'idProposicao':'0002',
       'dataHoraInicio':pd.to_datetime("2017-08-02 09:00:00.0"),
       'dataHoraFim':pd.to_datetime("2017-08-02 13:54:00.0"),
       'com_governo':1,
       'contra_governo':1,
  },
  {
       'siglaPartido':'PTB', 
       'voteId':'0002', 
       'idProposicao':'0002',
       'dataHoraInicio':pd.to_datetime("2017-08-02 09:00:00.0"),
       'dataHoraFim':pd.to_datetime("2017-08-02 13:54:00.0"),
       'com_governo':15,
       'contra_governo':3,
  },
  {
       'siglaPartido':'PODE', 
       'voteId':'0002', 
       'idProposicao':'0002',
       'dataHoraInicio':pd.to_datetime("2017-08-02 09:00:00.0"),
       'dataHoraFim':pd.to_datetime("2017-08-02 13:54:00.0"),
       'com_governo':9,
       'contra_governo':5,
  },
  {
       'siglaPartido':'PV', 
       'voteId':'0002', 
       'idProposicao':'0002',
       'dataHoraInicio':pd.to_datetime("2017-08-02 09:00:00.0"),
       'dataHoraFim':pd.to_datetime("2017-08-02 13:54:00.0"),
       'com_governo':3,
       'contra_governo':4,
  },
  {
       'siglaPartido':'REDE', 
       'voteId':'0002', 
       'idProposicao':'0002',
       'dataHoraInicio':pd.to_datetime("2017-08-02 09:00:00.0"),
       'dataHoraFim':pd.to_datetime("2017-08-02 13:54:00.0"),
       'com_governo':4,
       'contra_governo':0,
  },
  {
       'siglaPartido':'SD', 
       'voteId':'0002', 
       'idProposicao':'0002',
       'dataHoraInicio':pd.to_datetime("2017-08-02 09:00:00.0"),
       'dataHoraFim':pd.to_datetime("2017-08-02 13:54:00.0"),
       'com_governo':8,
       'contra_governo':6,
  },
  # Temer - 2
  {
      'siglaPartido':'DEM', 
       'voteId':'0003', 
       'idProposicao':'0003',
       'dataHoraInicio':pd.to_datetime("2017-10-25 09:00:00.0"),
       'dataHoraFim':pd.to_datetime("2017-10-25 14:23:00.0"),
       'com_governo':20,
       'contra_governo':9,
  },
  {
      'siglaPartido':'PCdoB', 
       'voteId':'0003', 
       'idProposicao':'0003',
       'dataHoraInicio':pd.to_datetime("2017-10-25 09:00:00.0"),
       'dataHoraFim':pd.to_datetime("2017-10-25 14:23:00.0"),
       'com_governo':0,
       'contra_governo':11,
  },
  {
      'siglaPartido':'PDT', 
       'voteId':'0003', 
       'idProposicao':'0003',
       'dataHoraInicio':pd.to_datetime("2017-10-25 09:00:00.0"),
       'dataHoraFim':pd.to_datetime("2017-10-25 14:23:00.0"),
       'com_governo':0,
       'contra_governo':17,
  },
      {
      'siglaPartido':'PATRI', 
       'voteId':'0003', 
       'idProposicao':'0003',
       'dataHoraInicio':pd.to_datetime("2017-10-25 09:00:00.0"),
       'dataHoraFim':pd.to_datetime("2017-10-25 14:23:00.0"),
       'com_governo':2,
       'contra_governo':0,
  },
  {
      'siglaPartido':'PHS', 
       'voteId':'0003', 
       'idProposicao':'0003',
       'dataHoraInicio':pd.to_datetime("2017-10-25 09:00:00.0"),
       'dataHoraFim':pd.to_datetime("2017-10-25 14:23:00.0"),
       'com_governo':1,
       'contra_governo':7,
  },
  {
      'siglaPartido':'MDB', 
       'voteId':'0003', 
       'idProposicao':'0003',
       'dataHoraInicio':pd.to_datetime("2017-10-25 09:00:00.0"),
       'dataHoraFim':pd.to_datetime("2017-10-25 14:23:00.0"),
       'com_governo':51,
       'contra_governo':7,
  },
  {
      'siglaPartido':'PP', 
       'voteId':'0003', 
       'idProposicao':'0003',
       'dataHoraInicio':pd.to_datetime("2017-10-25 09:00:00.0"),
       'dataHoraFim':pd.to_datetime("2017-10-25 14:23:00.0"),
       'com_governo':37,
       'contra_governo':6,
  },
  {
      'siglaPartido':'PPS',
       'voteId':'0003', 
       'idProposicao':'0003',
       'dataHoraInicio':pd.to_datetime("2017-10-25 09:00:00.0"),
       'dataHoraFim':pd.to_datetime("2017-10-25 14:23:00.0"),
       'com_governo':1,
       'contra_governo':8,
  },
  {
      'siglaPartido':'PR', 
       'voteId':'0003', 
       'idProposicao':'0003',
       'dataHoraInicio':pd.to_datetime("2017-10-25 09:00:00.0"),
       'dataHoraFim':pd.to_datetime("2017-10-25 14:23:00.0"),
       'com_governo':26,
       'contra_governo':10,
  },
  {
      'siglaPartido':'PRB', 
       'voteId':'0003', 
       'idProposicao':'0003',
       'dataHoraInicio':pd.to_datetime("2017-10-25 09:00:00.0"),
       'dataHoraFim':pd.to_datetime("2017-10-25 14:23:00.0"),
       'com_governo':16,
       'contra_governo':5,
  },
  {
      'siglaPartido':'PROS', 
       'voteId':'0003', 
       'idProposicao':'0003',
       'dataHoraInicio':pd.to_datetime("2017-10-25 09:00:00.0"),
       'dataHoraFim':pd.to_datetime("2017-10-25 14:23:00.0"),
       'com_governo':3,
       'contra_governo':3,
  },
  {
      'siglaPartido':'PSB', 
       'voteId':'0003', 
       'idProposicao':'0003',
       'dataHoraInicio':pd.to_datetime("2017-10-25 09:00:00.0"),
       'dataHoraFim':pd.to_datetime("2017-10-25 14:23:00.0"),
       'com_governo':11,
       'contra_governo':22,
  },
  {
      'siglaPartido':'PSC', 
       'voteId':'0003', 
       'idProposicao':'0003',
       'dataHoraInicio':pd.to_datetime("2017-10-25 09:00:00.0"),
       'dataHoraFim':pd.to_datetime("2017-10-25 14:23:00.0"),
       'com_governo':6,
       'contra_governo':4,
  },
  {
      'siglaPartido':'PSD', 
       'voteId':'0003', 
       'idProposicao':'0003',
       'dataHoraInicio':pd.to_datetime("2017-10-25 09:00:00.0"),
       'dataHoraFim':pd.to_datetime("2017-10-25 14:23:00.0"),
       'com_governo':20,
       'contra_governo':18,
  },
  {
      'siglaPartido':'PSDB', 
       'voteId':'0003', 
       'idProposicao':'0003',
       'dataHoraInicio':pd.to_datetime("2017-10-25 09:00:00.0"),
       'dataHoraFim':pd.to_datetime("2017-10-25 14:23:00.0"),
       'com_governo':20,
       'contra_governo':23,
  },
  {
      'siglaPartido':'PSL', 
       'voteId':'0003', 
       'idProposicao':'0003',
       'dataHoraInicio':pd.to_datetime("2017-10-25 09:00:00.0"),
       'dataHoraFim':pd.to_datetime("2017-10-25 14:23:00.0"),
       'com_governo':3,
       'contra_governo':0,
  },
  {
      'siglaPartido':'PSOL', 
       'voteId':'0003', 
       'idProposicao':'0003',
       'dataHoraInicio':pd.to_datetime("2017-10-25 09:00:00.0"),
       'dataHoraFim':pd.to_datetime("2017-10-25 14:23:00.0"),
       'com_governo':1,
       'contra_governo':4,
  },
  {
      'siglaPartido':'PT', 
       'voteId':'0003', 
       'idProposicao':'0003',
       'dataHoraInicio':pd.to_datetime("2017-10-25 09:00:00.0"),
       'dataHoraFim':pd.to_datetime("2017-10-25 14:23:00.0"),
       'com_governo':0,
       'contra_governo':57,
  },
  {
      'siglaPartido':'PTB', 
       'voteId':'0003', 
       'idProposicao':'0003',
       'dataHoraInicio':pd.to_datetime("2017-10-25 09:00:00.0"),
       'dataHoraFim':pd.to_datetime("2017-10-25 14:23:00.0"),
       'com_governo':14,
       'contra_governo':3,
  },
  {
      'siglaPartido':'AVANTE', 
       'voteId':'0003', 
       'idProposicao':'0003',
       'dataHoraInicio':pd.to_datetime("2017-10-25 09:00:00.0"),
       'dataHoraFim':pd.to_datetime("2017-10-25 14:23:00.0"),
       'com_governo':1,
       'contra_governo':3,
  },
  {
      'siglaPartido':'PODE', 
       'voteId':'0003', 
       'idProposicao':'0003',
       'dataHoraInicio':pd.to_datetime("2017-10-25 09:00:00.0"),
       'dataHoraFim':pd.to_datetime("2017-10-25 14:23:00.0"),
       'com_governo':8,
       'contra_governo':7,
  },
  {
      'siglaPartido':'PV', 
       'voteId':'0003', 
       'idProposicao':'0003',
       'dataHoraInicio':pd.to_datetime("2017-10-25 09:00:00.0"),
       'dataHoraFim':pd.to_datetime("2017-10-25 14:23:00.0"),
       'com_governo':3,
       'contra_governo':4,
  },
  {
      'siglaPartido':'REDE', 
       'voteId':'0003', 
       'idProposicao':'0003',
       'dataHoraInicio':pd.to_datetime("2017-10-25 09:00:00.0"),
       'dataHoraFim':pd.to_datetime("2017-10-25 14:23:00.0"),
       'com_governo':0,
       'contra_governo':4,
  },
  {
      'siglaPartido':'SD', 
       'voteId':'0003', 
       'idProposicao':'0003',
       'dataHoraInicio':pd.to_datetime("2017-10-25 09:00:00.0"),
       'dataHoraFim':pd.to_datetime("2017-10-25 14:23:00.0"),
       'com_governo':8,
       'contra_governo':13,
  },
]

In [11]:
for row in new_rows:
    df = df.append(row, ignore_index=True)

##### 2. Trabalhar com dados temporais

In [12]:
df = df.set_index("dataHoraFim")

In [13]:
# Essa função retorna um df resampleado e salva como csv,
# que serão os dados de base do D3.js
def resample_data(df, variables, interval):
    resampled_data = df[variables].resample(interval).mean().round(2)
    resampled_data = resampled_data.reset_index()
    return resampled_data

In [14]:
# Para plotar o gráfico no d3 de forma esteticamente agradável,
# vamos alterar a data de início e fim dos bins do resample.
# Os valores interiores não serão alteradas. O efeito é que
# os pontos não serão mais equidistantes no gráfico. Como, porém,
# não há votações nesse intervalo extendido, o formato da parte
# de dentro do gráfico não se altera.
def extend_date_limit(df):
    df_ = df.copy()
    
    # Preenche até o começo do ano da 1º vor
    year = str(df_.loc[0, 'dataHoraFim'].year)
    df_.loc[0, 'dataHoraFim'] = pd.to_datetime( str(year) + '-01-01')
    
    # Preenche a última data
    df_.loc[df_.shape[0] - 1, 'dataHoraFim'] = pd.to_datetime('2019-01-01')
    
    return df_

In [15]:
# Salva um para cada partido
for party in df.siglaPartido.unique():
    df_ = df.copy()
    df_ = df_[df_.siglaPartido == party]
    df_ = resample_data(df_, ['com_governo', 'contra_governo'], '6M')
    df_ = extend_date_limit(df_)
    df_.to_csv("../viz/data/" + party.lower() + '6m' + '.csv', index=False)

In [16]:
# Salva um para cada partido, sem resample.
for party in df.siglaPartido.unique():
    df_ = df.copy()
    df_ = df_[df_.siglaPartido == party]
    df_ = df_.reset_index()
    df_.to_csv("../viz/data/" + party.lower() +'-barras' + '.csv', index=False)

In [21]:
# Salva um arquivo geral
df_ = df.groupby(["voteId", "dataHoraFim"]).sum().reset_index()

# Adiciona placares da PEC da CPMF, que também não constam dos dados da API da Câmara
cpmf = [
    {'vote_id':'0004', 'dataHoraFim':pd.to_datetime("2007-09-20 00:00:00"), 'com_governo':338, 'contra_governo':119}, 
    {'vote_id':'0005', 'dataHoraFim':pd.to_datetime("2007-10-10 00:00:00"), 'com_governo':333, 'contra_governo':115}
]

df_ = df_.append(cpmf)
df_ = df_.reset_index().set_index("dataHoraFim").resample("6M").mean().reset_index()
df_ = extend_date_limit(df_)
df_.to_csv("../viz/data/todos6m.csv")

In [18]:
# Salva o arquivo com os dados gerais de todos os partidos, sem resample
df_ = df.groupby(["voteId", "dataHoraFim"]).sum().reset_index()

# Adiciona placares da PEC da CPMF, que também não constam dos dados da API da Câmara
cpmf = [
    {'voteId':'0004', 'dataHoraFim':pd.to_datetime("2007-09-20 00:00:00"), 'com_governo':338, 'contra_governo':119}, 
    {'voteId':'0005', 'dataHoraFim':pd.to_datetime("2007-10-10 00:00:00"), 'com_governo':333, 'contra_governo':115}
]

df_ = df_.append(cpmf, ignore_index=True)

df_.to_csv("../viz/data/todos-barras.csv", index=False)